In [ ]:
!pip install rouge-score
!pip install datasets

In [ ]:
from rouge_score import rouge_scorer

In [ ]:
from datasets import load_dataset, load_from_disk, Dataset
import numpy as np
import pandas as pd

In [ ]:
data_base = 'train_data/'

# dataset_name = 'cnn_dailymail'

data_path = data_base#+'/'+dataset_name

In [ ]:

train_data = load_from_disk(
    'train/'+data_path
)

# train_data = dataset['train']
# test_data = dataset['test']
# val_data = dataset['validation']

In [ ]:
train_data

Dataset({
    features: ['article', 'highlights', 'id', 'article_sentences', 'article_length', 'article_sentence_max_length'],
    num_rows: 287113
})

In [ ]:
# eng_stopwords = stopwords.words("english")

## get rouge scores for individual sentences

In [ ]:
def get_rouge_scores(article):
  summary = article['highlights']
  sentence_scores = []
  sentences = article['article_sentences']
  for sentence in sentences:
    scores = scorer.score(summary, sentence)
    sentence_scores.append(np.mean([v.fmeasure for k,v in scores.items()]))
  article['article_rouge_f1'] = sentence_scores
  return article

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
# s = train_data['article_sentences'][0]
# highl = train_data['highlights'][0]

In [ ]:
# new_scores = []
# for sen in s:
#   new_scores.append(np.mean([v.fmeasure for k,v in scorer.score(highl, sen).items()]))

In [ ]:
# train_scores = train_data.map(
#     get_rouge_scores, 
#     batched=False, 
#     remove_columns=['article', 'highlights', 'id', 'article_sentences', 'article_length', 'article_sentence_max_length']
# )

In [ ]:
# train_scores.save_to_disk(data_path+'/sentences/train/scores')

In [ ]:
# train_scores

## top 10 sentences and generate candidate summaries

In [ ]:
# import itertools
# def select_top_p_sent(article):
#   scores = article['article_rouge_f1']
#   sent = sorted(np.argsort(scores)[-10:])
#   article['article_top_sent'] = sent
#   summaries = []
#   for summary in itertools.combinations(sent, 4):
#     summaries.append(summary)
#   article['candidate_summaries'] = summaries
#   return article

In [ ]:
# train_sentence_idx = train_scores.map(
#     select_top_p_sent, 
#     batched=False
# )

In [ ]:
# train_sentence_idx.save_to_disk(data_path+'/sentences/train/candidate_summaries')

In [ ]:
all_scores = train_scores['article_rouge_f1']

In [ ]:
all_top_sentences = []
for scores in all_scores:
  top_sentences = sorted(np.argsort(scores)[-10:])
  all_top_sentences.append(top_sentences)

In [ ]:
import pickle

with open(data_path+'/sentences/train/top_sentences.pkl', 'wb') as pickle_file:
    pickle.dump(all_top_sentences, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import itertools

all_summaries = []
for top_sentences in all_top_sentences:
  summaries = []
  for summary in itertools.combinations(top_sentences, 4):
    summaries.append(summary)
  all_summaries.append(summaries)

In [ ]:
with open(data_path+'/sentences/train/all_top_summaries.pkl', 'wb') as pickle_file:
    pickle.dump(all_summaries, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

## Select top k summaries and generate gold actions

In [ ]:
import pickle

In [ ]:
file = open("train/all_top_summaries.pkl", 'rb')

# dump information to that file
all_summaries = pickle.load(file)

# close the file
file.close()

In [ ]:
file = open("top_sentences.pkl", 'rb')

# dump information to that file
top_sentences = pickle.load(file)

# close the file
file.close()

In [ ]:
top_sentences[0]

[2, 3, 9, 17, 31, 67, 70, 76, 78, 82]

In [ ]:
len(all_summaries)

287113

In [ ]:
train_data['article_sentences'][0]

["It's official: U",
 'President Barack Obama wants lawmakers to weigh in on whether to use military force in Syria',
 'Obama sent a letter to the heads of the House and Senate on Saturday night, hours after announcing that he believes military action against Syrian targets is the right step to take over the alleged use of chemical weapons',
 'The proposed legislation from Obama asks Congress to approve the use of military force "to deter, disrupt, prevent and degrade the potential for future uses of chemical weapons or other weapons of mass destruction',
 '" It\'s a step that is set to turn an international crisis into a fierce domestic political battle',
 'There are key questions looming over the debate: What did U',
 'weapons inspectors find in Syria? What happens if Congress votes no? And how will the Syrian government react? In a televised address from the White House Rose Garden earlier Saturday, the president said he would take his case to Congress, not because he has to -- but 

In [ ]:
all_highlights = train_data['highlights']
all_sentences = train_data['article_sentences']

In [ ]:
articles = train_data.num_rows

In [ ]:
all_summary_scores = []
all_top_summaries = []
all_gold_actions = []

for article_no in range(articles):
    target_summary = all_highlights[article_no]
    article_sentences = all_sentences[article_no]
    candidate_summaries = all_summaries[article_no]

    candidate_scores = []
    for summary in candidate_summaries:
        candidate_summary = " ".join([article_sentences[i] for i in summary])
        scores = scorer.score(target_summary, candidate_summary)
        candidate_scores.append(np.mean([v.fmeasure for k,v in scores.items()]))
    all_summary_scores.append(candidate_scores)
    top_summaries = [candidate_summaries[i] for i in np.argsort(candidate_scores)[-10:]]
    all_top_summaries.append(top_summaries)

    gold_actions= []
    for top_summary in top_summaries:
        actions = []
        for i in top_sentences[0]:
            if i in top_summary:
                actions.append(1)
            else:
                actions.append(2)
        gold_actions.append(actions)
    all_gold_actions.append(gold_actions)

In [ ]:
f = open('train/all_top_summaries.pkl', 'rb')

all_top_summaries = pickle.load(f)
f.close()

In [ ]:
all_top_summaries[0]

[(9, 17, 31, 82),
 (3, 9, 17, 82),
 (2, 9, 31, 82),
 (2, 17, 70, 82),
 (2, 17, 78, 82),
 (2, 17, 76, 82),
 (2, 3, 17, 82),
 (2, 17, 67, 82),
 (2, 17, 31, 82),
 (2, 9, 17, 82)]

In [ ]:
all_gold_actions = []
for article_no in range(articles):
    gold_actions= []
    top_summaries = all_top_summaries[article_no]
    for top_summary in top_summaries:
        actions = []
        for i in top_sentences[article_no]:
            if i in top_summary:
                actions.append(1)
            else:
                actions.append(2)
        gold_actions.append(actions)
    all_gold_actions.append(gold_actions)

In [ ]:
# all_gold_actions[10]

In [ ]:
# len(all_summary_scores[0]), all_top_summaries[0][0]

In [ ]:
all_gold_actions[1]

[[1, 2, 2, 1, 2, 1, 2, 2, 1, 2],
 [1, 2, 2, 2, 2, 1, 1, 2, 1, 2],
 [1, 2, 1, 2, 2, 1, 2, 2, 1, 2],
 [1, 2, 2, 2, 2, 1, 2, 1, 1, 2],
 [1, 1, 2, 2, 1, 2, 2, 2, 1, 2],
 [1, 2, 2, 2, 1, 2, 1, 2, 1, 2],
 [1, 2, 2, 1, 1, 2, 2, 2, 1, 2],
 [1, 2, 1, 2, 1, 2, 2, 2, 1, 2],
 [1, 2, 2, 2, 1, 2, 2, 1, 1, 2],
 [1, 2, 2, 2, 1, 1, 2, 2, 1, 2]]

In [ ]:
# with open('all_summary_scores.pkl', 'wb') as pickle_file:
#     pickle.dump(all_summary_scores, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)
    
# with open('all_top_summaries.pkl', 'wb') as pickle_file:
#     pickle.dump(all_top_summaries, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

with open('all_gold_actions.pkl', 'wb') as pickle_file:
    pickle.dump(all_gold_actions, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

## Get Rewards for each action sequence

In [ ]:
import pickle

In [ ]:
file= open('all_gold_actions.pkl', 'rb')
all_gold_actions = pickle.load(file)
file.close()

In [ ]:
file = open("train/top_sentences.pkl", 'rb')

# dump information to that file
top_sentences = pickle.load(file)

# close the file
file.close()

In [ ]:
all_gold_actions[1], dataset['all_gold_actions']['all_gold_actions'][1]

([[1, 2, 2, 1, 2, 1, 2, 2, 1, 2],
  [1, 2, 2, 2, 2, 1, 1, 2, 1, 2],
  [1, 2, 1, 2, 2, 1, 2, 2, 1, 2],
  [1, 2, 2, 2, 2, 1, 2, 1, 1, 2],
  [1, 1, 2, 2, 1, 2, 2, 2, 1, 2],
  [1, 2, 2, 2, 1, 2, 1, 2, 1, 2],
  [1, 2, 2, 1, 1, 2, 2, 2, 1, 2],
  [1, 2, 1, 2, 1, 2, 2, 2, 1, 2],
  [1, 2, 2, 2, 1, 2, 2, 1, 1, 2],
  [1, 2, 2, 2, 1, 1, 2, 2, 1, 2]],
 [[2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
  [2, 2, 2, 1, 2, 2, 2, 2, 2, 2]])

In [ ]:
all_highlights = train_data['highlights']
all_sentences = train_data['article_sentences']

In [ ]:
top_full_sentences = []
for article_no in range(len(top_sentences)):
    top_full_sentences.append([all_sentences[article_no][sent_no] for sent_no in top_sentences[article_no]])

In [ ]:
top_full_sentences[1][0]

"-- Usain Bolt rounded off the world championships Sunday by claiming his third gold in Moscow as he anchored Jamaica to victory in the men's 4x100m relay"

In [ ]:
all_gold_rewards = []
for article_no in range(len(all_gold_actions)):
    action_paths = all_gold_actions[article_no]
    all_rewards = []
    for action_path in action_paths:
        curr_sum = []
        rewards = []
        for sent_no in range(len(action_path)):
            if action_path[sent_no] == 1:
                # keep the sent
                curr_sum.append(top_full_sentences[article_no][sent_no])
                scores = scorer.score(all_highlights[article_no], " ".join(curr_sum))
                rewards.append(np.mean([v.fmeasure for k,v in scores.items()]))
            else:
                rewards.append(0)# skipping the sentence no reward
        all_rewards.append(rewards)
    if article_no %10000 == 0:
        print("done article:", article_no)
    all_gold_rewards.append(all_rewards)

done article: 0


In [ ]:
all_gold_rewards[10][1], all_gold_actions[10][1]

([0,
  0,
  0.3785220627325891,
  0.4142313903214044,
  0,
  0,
  0.5252818035426731,
  0.5939281861611959,
  0,
  0],
 [2, 2, 1, 1, 2, 2, 1, 1, 2, 2])

In [ ]:
all_gold_rewards[10][1], all_gold_actions[10][1]

([0,
  0,
  0.3785220627325891,
  0.4142313903214044,
  0,
  0,
  0.5252818035426731,
  0.5939281861611959,
  0,
  0],
 [2, 2, 1, 1, 2, 2, 1, 1, 2, 2])

In [ ]:
train_data['id'][1]

In [ ]:
from datasets import DatasetDict

dataset = DatasetDict({
    "top_full_sentences": Dataset.from_pandas(pd.DataFrame({"top_full_sentences": top_full_sentences})),
    "top_sentences_index": Dataset.from_pandas(pd.DataFrame({"top_sentences_index": top_sentences})),
    "highlights": Dataset.from_pandas(pd.DataFrame({"highlights": all_highlights})),
    "all_gold_actions": Dataset.from_pandas(pd.DataFrame({"all_gold_actions": all_gold_actions})),
    "all_gold_rewards": Dataset.from_pandas(pd.DataFrame({"all_gold_rewards": all_gold_rewards})),
    "article_id": Dataset.from_pandas(pd.DataFrame({"article_id": train_data['id']}))
})

In [ ]:
data_path

'train_data/'

In [ ]:
dataset.save_to_disk(data_path+'/train_rewards_new')

In [ ]:
# dataset = load_from_disk(
#     data_path+'/train_rewards_new'
# )

In [ ]:
dataset['all_gold_rewards']['all_gold_rewards'][1]

[[0.40784832451499115,
  0.0,
  0.0,
  0.2783589478668905,
  0.0,
  0.24320987654320989,
  0.0,
  0.0,
  0.23002710027100273,
  0.0],
 [0.40784832451499115,
  0.0,
  0.0,
  0.0,
  0.0,
  0.321962481962482,
  0.2427383135347737,
  0.0,
  0.23511904761904764,
  0.0],
 [0.40784832451499115,
  0.0,
  0.27767499229108855,
  0.0,
  0.0,
  0.24999358727715784,
  0.0,
  0.0,
  0.23653846153846156,
  0.0],
 [0.40784832451499115,
  0.0,
  0.0,
  0.0,
  0.0,
  0.321962481962482,
  0.0,
  0.2640258366915316,
  0.23735237087960512,
  0.0],
 [0.40784832451499115,
  0.2771241830065359,
  0.0,
  0.0,
  0.2629010281184194,
  0.0,
  0.0,
  0.0,
  0.23807303807303812,
  0.0],
 [0.40784832451499115,
  0.0,
  0.0,
  0.0,
  0.3348098929494278,
  0.0,
  0.25218579234972677,
  0.0,
  0.23916373794719292,
  0.0],
 [0.40784832451499115,
  0.0,
  0.0,
  0.2783589478668905,
  0.26407144054202875,
  0.0,
  0.0,
  0.0,
  0.2394843962008141,
  0.0],
 [0.40784832451499115,
  0.0,
  0.27767499229108855,
  0.0,
  0.264

In [ ]:
# target_article = 
dataset['all_gold_actions']['all_gold_actions'][1]

In [ ]:
dataset['article_id']['article_id'][0:10]

In [ ]:
#old_target_article

[[2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 2, 2, 2, 2, 2, 2, 2],
 [2, 2, 2, 1, 2, 2, 2, 2, 2, 2]]

In [ ]:
# def get_top_summaries(article, article_no, all_summaries = all_summaries):
#     target_summary = article['highlights']
#     all_sentences = article['article_sentences']
#     candidate_summaries = all_summaries[article_no]
    
#     candidate_scores = []
#     for summary in candidate_summaries:
#         candidate_summary = " ".join([all_sentences[i] for i in summary])
#         scores = scorer.score(target_summary, candidate_summary)
#         candidate_scores.append(np.mean([v.fmeasure for k,v in scores.items()]))
#     top_summaries = [candidate_summaries[i] for i in np.argsort(candidate_scores)[-10:]]
#     gold_actions= []
#     for top_summary in top_summaries:
#         actions = []
#         for i in top_sentences[0]:
#             if i in top_summary:
#                 actions.append(1)
#             else:
#                 actions.append(2)
#         gold_actions.append(actions)
#     article['gold_actions'] = gold_actions
#     return article

In [ ]:
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# train_sum_scores = train_data.map(
#     get_top_summaries, 
#     batched=False,
#     remove_columns=['highlights', 'id', 'article_sentences', 'article_length', 'article_sentence_max_length'],
#     with_indices=True
# )

In [ ]:
train_sum_scores

In [ ]:
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# article_scores = []
# for article_no in range(0,train_data.num_rows):
#   summary = train_data['highlights'][article_no]
#   sentence_scores = []
#   sentences = train_data['article_sentences'][article_no]
#   for sentence in sentences:
#     # sentence = " ".join([word for word in sentence if word not in eng_stopwords])
#     scores = scorer.score(sentence, summary)
#     sentence_scores.append(np.mean([v.fmeasure for k,v in scores.items()]))
#   article_scores.append(sentence_scores)

In [ ]:
# article_no

In [ ]:
# import pickle

# with open(data_path+'/sentences/train/rouge_scores_338.pkl', 'wb') as pickle_file:
#     pickle.dump(article_scores, pickle_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
# from rouge_score import rouge_scorer

# scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
# scores = scorer.score('The quick brown fox jumps over the lazy dog',
#                       'The quick brown dog jumps on the log.')
# np.mean([v.fmeasure for k,v in scores.items()])

In [ ]:
# scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=False)
# scores = scorer.score('The quick brown fox jumps over the lazy dog',
#                       'The quick brown dog jumps on the log.')
# scores